`Import the necessary library`

In [19]:
import pickle
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob

`Get the Camera calibration matrix from calibratrion images`

In [20]:
def calibratematrix():
    objpoints = []
    imgpoints = []
    outimages = []
    originalImages = []
    objp = np.zeros((6*9,3), np.float32) 
    objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2) # x and y cordinates.
    calibrationImages = list(map(lambda imageFileName: (imageFileName, cv2.imread(imageFileName)), glob.glob('./camera_cal/c*.jpg')))

    for imageAndFile in calibrationImages:
        fileName, image = imageAndFile
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        ret, corners = cv2.findChessboardCorners(gray, (9,6), None)
        if ret == True:
            imgpoints.append(corners)
            objpoints.append(objp)

            img_points = cv2.drawChessboardCorners(image.copy(), (9,6), corners, ret)
            outimages.append(img_points)
            originalImages.append(image)
            
    img_2 = originalImages[1]
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_2.shape[0:2], None, None)
    return mtx, dist

`Function to undistort the image`

In [21]:
def undistort(img,mtx, dist):
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    return undist

In [22]:
def histogramEqualization(img) :
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    cl1 = clahe.apply(img)
    return cl1

`Test to check undistortion`

In [23]:
dist_img = cv2.imread("camera_cal/calibration1.jpg")
mtx, dist =calibratematrix()
undist_img = undistort(dist_img, mtx, dist)
cv2.imshow("undistorted", undist_img)
cv2.waitKey(5000)
cv2.destroyAllWindows()

In [25]:
def clahe (bgr) :
    lab = cv2.cvtColor(bgr, cv2.COLOR_BGR2LAB)
    lab_planes = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0,tileGridSize=(8,8))
    lab_planes[0] = clahe.apply(lab_planes[0])
    lab = cv2.merge(lab_planes)
    bgr = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)
    return bgr

`Implemented edge detection using gradient and  hls transform`

In [26]:
def edge_detection(img, s_thresh=(170, 255), sx_thresh=(20, 100)):
    img = np.copy(img)
    # Convert to HLS color space and separate the V channel
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    lab = cv2.cvtColor(img, cv2.COLOR_RGB2Lab)
    luv = cv2.cvtColor(img, cv2.COLOR_RGB2Luv)
    
    l_luv_channel = luv[:,:,0]
    
    b_lab_channel = lab[:,:,2]
    
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]
    
    
    l_luv_binary = np.zeros_like(l_luv_channel)
    l_luv_binary[(l_luv_channel >= 230) & (l_luv_channel <= s_thresh[1])] = 1
    
    b_lab_binary = np.zeros_like(b_lab_channel)
    b_lab_binary[(b_lab_channel >= s_thresh[0]) & (b_lab_channel <= s_thresh[1])] = 1
    
    
    # Sobel x
    sobelx = cv2.Sobel(l_channel, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1
    
    # Threshold color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    # Stack each channel
    
    s_sx_mask = cv2.bitwise_and(s_binary, sxbinary)
    color_binary = np.dstack(( l_luv_binary, b_lab_binary, s_sx_mask)) * 255
    
    gray_binary = np.zeros_like(s_channel)    
    gray_binary[l_luv_binary[:,:] == 1] = 255
    gray_binary[b_lab_binary[:,:] == 1] = 255
    gray_binary[s_sx_mask[:,:] == 1] = 255

    
    #gray_binary = cv2.cvtColor(color_binary, cv2.COLOR_BGR2GRAY)
    #gray_binary[gray_binary[:,:] >= 80] = 255
    return gray_binary,color_binary

```warpPerspective``` transforms image to bird- eye view

In [27]:
def warpPerspective(img, order):
    img_size = (img.shape[1],img.shape[0])
    src = np.float32([[585, 460],[203, 720],[1127, 720],[695, 460]])
    dst = np.float32([[320, 0],[320, 720],[960, 720],[960, 0]])
    if (order == 1) :
        M = cv2.getPerspectiveTransform(src, dst)
    if (order == -1) :
        M = cv2.getPerspectiveTransform(dst, src)
    warped = cv2.warpPerspective(img, M, img_size)
    return warped

`find_lane_pixels` uses image histogram to find the left and right lane base positions. Once the lane base is determined, sliding window algorithm is implemented to find the points on the lane. `fit_polynomial` finds the coefficients of the left and right curve

In [28]:
def find_lane_pixels(binary_warped):
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    # Create an output image to draw on and visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # HYPERPARAMETERS
    # Choose the number of sliding windows
    nwindows = 9
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),
        (win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),
        (win_xright_high,win_y_high),(0,255,0), 2) 
        
        # Identify the nonzero pixels in x and y within the window #
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty, out_img


def fit_polynomial(binary_warped,left_fit, right_fit):
    # Find our lane pixels first
    leftx, lefty, rightx, righty, out_img = find_lane_pixels(binary_warped)

    # Fit a second order polynomial to each using `np.polyfit`
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)

    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    try:
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty

    return out_img,left_fit,right_fit

In [79]:
def measure_curvature_real(ploty, left_fit_cr, right_fit_cr):
    '''
    Calculates the curvature of polynomial functions in meters.
    '''
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    
    y_eval = np.max(ploty)
    
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*xm_per_pix/ym_per_pix + left_fit_cr[1]*xm_per_pix/ym_per_pix)**2)**1.5) / np.absolute(2*left_fit_cr[0]*xm_per_pix/(ym_per_pix*ym_per_pix))
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*xm_per_pix/ym_per_pix + right_fit_cr[1]*xm_per_pix/ym_per_pix)**2)**1.5) / np.absolute(2*right_fit_cr[0]*xm_per_pix/(ym_per_pix*ym_per_pix))
    return left_curverad, right_curverad

```findOffset``` find the position of camera with respect to the lane

In [74]:
def findOffset(img,left_fit_m, right_fit_m):
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    xMax = img.shape[1]
    yMax = img.shape[0]
    vehicleCenter = xMax / 2
    lineLeft = left_fit_m[0]*yMax**2 + left_fit_m[1]*yMax + left_fit_m[2]
    lineRight = right_fit_m[0]*yMax**2 + right_fit_m[1]*yMax + right_fit_m[2]
    lineMiddle = lineLeft + (lineRight - lineLeft)/2
    diffFromVehicle = (lineMiddle - vehicleCenter)*xm_per_pix
    if diffFromVehicle > 0:
        message = '{:.2f} m right'.format(diffFromVehicle)
    else:
        message = '{:.2f} m left'.format(-diffFromVehicle)
        
    font = cv2.FONT_HERSHEY_SIMPLEX
    fontColor = (255, 255, 255)
    fontScale              = 1
    fontColor              = (0,255,0)
    lineType               = 2
   
    cv2.putText(img, 'Vehicle is {} of center'.format(message), (70, 130), font, fontScale, fontColor, lineType)
    return img
    

Once the lane curve is identified from the first image,
curvature coefficents are passed on to necxt image and are utilized by the ```search_around_poly``` function,
to identify the points on the lane in the new image.
``` fit_poly``` returns new coeffecients for the the current image which is passed to next image.

In [31]:
def fit_poly(img_shape, leftx, lefty, rightx, righty):
    ### TO-DO: Fit a second order polynomial to each with np.polyfit() ###
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    # Generate x and y values for plotting
    ploty = np.linspace(0, img_shape[0]-1, img_shape[0])
    ### TO-DO: Calc both polynomials using ploty, left_fit and right_fit ###
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    return left_fitx, right_fitx, ploty, left_fit, right_fit

def search_around_poly(binary_warped,left_fit, right_fit):
   
    margin = 100

    # Grab activated pixels
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
  
    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + 
                    left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + 
                    left_fit[1]*nonzeroy + left_fit[2] + margin)))
    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + 
                    right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + 
                    right_fit[1]*nonzeroy + right_fit[2] + margin)))
    
    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    # Fit new polynomials
    left_fitx, right_fitx, ploty,left_fit, right_fit = fit_poly(binary_warped.shape, leftx, lefty, rightx, righty)
    
    ## Visualization ##
    # Create an image to draw on and an image to show the selection window
    left_curverad, right_curverad = measure_curvature_real(ploty, left_fit, right_fit)
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    result = np.zeros_like(out_img)
   
    line_window1 = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx, 
                              ploty])))])
    line_pts = np.hstack((line_window1, line_window2))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(result, np.int_([line_pts]), (0,255, 0))
    return result,left_fit, right_fit, left_curverad, right_curverad

```radiusCurvature``` prints the mean of the right and left curvature on the image.

In [64]:
def radiusCurvature(img, left_curverad, right_curverad):
        
    mean_curvature = ((left_curverad + right_curverad)/2)
    message = 'Left R : {:.0f} m, Right R : {:.0f} m '.format(left_curverad, right_curverad)
    text = '{} '.format(message)
    #text = 'Curvature = ' + str(mean_curvature)+' m'
    font                   = cv2.FONT_HERSHEY_SIMPLEX
    position = (70,70)
    fontScale              = 1
    fontColor              = (0,255,0)
    lineType               = 2

    cv2.putText(img,text, position,font, fontScale,fontColor,lineType)
    
    return img
    

```weighted_img``` merges initial image with the augmented lane detected image.

In [33]:
def weighted_img(img, initial_img, α=1.0, β=0.3, γ=0.):
    """
   
    The result image is computed as follows:
    
    initial_img * α + img * β + γ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, γ)
    

In [82]:
mtx, dist =calibratematrix()
count = 1
left_fit = []
right_fit = []
path = 'output_images/'

def save_output_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image where lines are drawn on lanes)
    global mtx, dist, count,left_fit,right_fit
    undist = undistort(image, mtx, dist)
    clahe_img = clahe(undist)
    edge_image,color_binary = edge_detection(clahe_img)
    wrap_undist = warpPerspective(undist, 1)
    warped_edge_img = warpPerspective(edge_image, 1)
    if (count == 1):
        edge,left_fit,right_fit = fit_polynomial(warped_edge_img,left_fit, right_fit)
    lane_img,left_fit, right_fit, left_curverad, right_curverad = search_around_poly(warped_edge_img,left_fit,right_fit)
    unwrap_image = warpPerspective(lane_img, -1)
    
    radius_curve_img = radiusCurvature(unwrap_image, left_curverad, right_curverad)
    offset_img = findOffset(radius_curve_img,left_fit, right_fit)
    result = weighted_img(offset_img, undist)
    
    cv2.imwrite(path+"undist"+".jpg", undist)
    cv2.imwrite(path+"edge_image"+".jpg", edge_image)
    cv2.imwrite(path+"wrap_undist"+".jpg", wrap_undist)
    cv2.imwrite(path+"warped_edge_img"+".jpg", warped_edge_img)
    cv2.imwrite(path+"lane_img"+".jpg", lane_img)
    cv2.imwrite(path+"unwrap_image"+".jpg", unwrap_image)
    cv2.imwrite(path+"radius_curve_img"+".jpg", radius_curve_img)
    cv2.imwrite(path+"offset_img"+".jpg", offset_img) 
    cv2.imwrite(path+"result"+".jpg", result)
    

In [83]:
vidcap = cv2.VideoCapture("project_video.mp4")
success,image = vidcap.read()
val = 1
success = True
while success:
    success,image = vidcap.read()
    if val == 350:                   
        save_output_image(image)
    val += 1

``` Libraries for video processing.```

In [36]:
# Import everything needed to edit/save/watch video clips
#import imageio
#import imageio.plugins.ffmpeg.download()
from IPython.display import HTML
from moviepy.editor import VideoFileClip

In [75]:
mtx, dist =calibratematrix()
count = 1
left_fit = []
right_fit = []

def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image where lines are drawn on lanes)
    global mtx, dist, count,left_fit,right_fit
    undist = undistort(image, mtx, dist)
    clahe_img = clahe(undist)
    edge_image,color_binary = edge_detection(clahe_img)
    warped_edge_img = warpPerspective(edge_image, 1)
    if (count == 1):
        edge,left_fit,right_fit = fit_polynomial(warped_edge_img,left_fit, right_fit)
    lane_img,left_fit, right_fit, left_curverad, right_curverad = search_around_poly(warped_edge_img,left_fit,right_fit)
    unwrap_image = warpPerspective(lane_img, -1)
    
    radius_curve_img = radiusCurvature(unwrap_image, left_curverad, right_curverad)
    offset_img = findOffset(radius_curve_img,left_fit, right_fit)
    result = weighted_img(offset_img, undist)
   
    return result

In [76]:
dist_img = cv2.imread("test_images/straight_lines1.jpg")
undist_img = process_image(dist_img)
cv2.imshow("undistorted", undist_img)
cv2.waitKey(5000)
cv2.destroyAllWindows()

In [80]:
white_output = 'output_videos/project_video.mp4'
clip1 = VideoFileClip("project_video.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

t:   0%|          | 3/1260 [00:00<00:56, 22.09it/s, now=None]

Moviepy - Building video output_videos/project_video.mp4.
Moviepy - Writing video output_videos/project_video.mp4



Moviepy - Done !
Moviepy - video ready output_videos/project_video.mp4
CPU times: user 2min 45s, sys: 2.28 s, total: 2min 47s
Wall time: 1min 15s


In [81]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

t:   1%|          | 3/485 [00:00<00:17, 27.38it/s, now=None]

Moviepy - Building video output_videos/challenge_video.mp4.
Moviepy - Writing video output_videos/challenge_video.mp4



t:   1%|▏         | 7/485 [00:00<00:21, 22.04it/s, now=None]

TypeError: expected non-empty vector for x

t:   1%|▏         | 7/485 [00:19<00:21, 22.04it/s, now=None]

In [ ]:

""".format(white_output))